In [1]:
import nltk
import torch
from transformers import T5Tokenizer, AutoModelForSeq2SeqLM
from peft import PeftConfig, PeftModel
from t5.dataset import load_spider_datasets
from t5.inference import inference, evaluate_result

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
print(torch.cuda.is_available())

True


In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# parameters

tokenizer_name = 't5-base'
base_model_name = "RoxyRong/t5_base_finetuned"
peft_model_name = "RoxyRong/t5_base_soft_prompt_new"
result_path = f'results/predicted_result_t5_base_soft_prompt_new.txt'

eval_soft_prompt = True

In [5]:
# evaluate 

tokenizer = T5Tokenizer.from_pretrained(tokenizer_name, model_max_length=512)
model = AutoModelForSeq2SeqLM.from_pretrained(base_model_name)

# if eval_soft_prompt:
peft_model_id = peft_model_name
config = PeftConfig.from_pretrained(peft_model_id)
model = PeftModel.from_pretrained(model, peft_model_id)

model = model.to("cuda")

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [6]:
# load dataset
_, _, dev_spider = load_spider_datasets()

In [7]:
# inference
inference(dev_spider, model, tokenizer, result_path)

0
100
200
300
400
500
600
700
800
900
1000


In [8]:
# evaluate 
evaluate_result(result_path)

('                     easy                 medium               '
 'hard                 extra                all                 \n'
 'count                250                  440                  '
 '174                  170                  1034                \n'
 '=====================   EXECUTION ACCURACY     =====================\n'
 'execution            0.668                0.416                '
 '0.322                0.182                0.423               \n'
 '\n'
 '====================== EXACT MATCHING ACCURACY =====================\n'
 'exact match          0.720                0.425                '
 '0.316                0.141                0.431               \n'
 '\n'
 '---------------------PARTIAL MATCHING ACCURACY----------------------\n'
 'select               0.960                0.920                '
 '0.977                0.864                0.936               \n'
 'select(no AGG)       0.975                0.932                '
 '0.988                0

In [9]:
import subprocess
import pprint

eval_path = f"third_party/spider/evaluation.py"
gold = f"third_party/spider/evaluation_examples/gold_example.txt"
pred = result_path
db_dir = f"spider/database"
table = f"spider/tables.json"
etype = "all"

cmd_str = f"python3 \"{eval_path}\" --gold \"{gold}\" --pred \"{pred}\" --db \"{db_dir}\" --table \"{table}\" --etype {etype} "
result = subprocess.run(cmd_str, shell=True, capture_output=True, text=True)
pprint.pprint(result.stdout[-4633:])

('                     easy                 medium               '
 'hard                 extra                all                 \n'
 'count                250                  440                  '
 '174                  170                  1034                \n'
 '=====================   EXECUTION ACCURACY     =====================\n'
 'execution            0.668                0.416                '
 '0.322                0.182                0.423               \n'
 '\n'
 '====================== EXACT MATCHING ACCURACY =====================\n'
 'exact match          0.720                0.425                '
 '0.316                0.141                0.431               \n'
 '\n'
 '---------------------PARTIAL MATCHING ACCURACY----------------------\n'
 'select               0.960                0.920                '
 '0.977                0.864                0.936               \n'
 'select(no AGG)       0.975                0.932                '
 '0.988                0